# Custom Git Repo with Python

In [1]:
import hashlib
import zlib
import os
import datetime

In [2]:
# remove the files if we've already done this; otherwise start with clean directory structure
try:
    os.mkdir('gitsimplepython/')
except FileExistsError:
    os.system('rm -rf gitsimplepython/.git')

os.mkdir('gitsimplepython/.git/')
os.mkdir('gitsimplepython/.git/objects')

In [3]:
!find gitsimplepython

gitsimplepython
gitsimplepython/.git
gitsimplepython/.git/objects


### What's needed for a Git file?

1. content
2. header information
3. blob `<content-length>\0\<content>\n`

### Create the file content

In [4]:
blob_content = b"hello world"
blob_content

b'hello world'

### Create the header information

In [5]:
blob_header = "blob {}\0".format(len(blob_content)).encode('utf-8')
blob_header

b'blob 11\x00'

### Create the blob and a hash

In [6]:
blob_store = blob_header + blob_content + b"\n"
print(blob_store)

b'blob 11\x00hello world\n'


In [7]:
hash_object = hashlib.sha1(blob_store)
blob_hash = hash_object.hexdigest()
blob_hash

'9972de6ab4626b28c8798abe3624bec5c82ea9c4'

### Compress the file

In [8]:
zlib_blob_store = zlib.compress(blob_store)
zlib_blob_store

b'x\x9cK\xca\xc9OR04d\xc8H\xcd\xc9\xc9W(\xcf/\xcaI\xe1\x02\x00D\x03\x06\x88'

### This file needs a directory

In [9]:
blob_directoryname = 'gitsimplepython/.git/objects/{}/'.format(blob_hash[0:2])
blob_directoryname

'gitsimplepython/.git/objects/99/'

In [10]:
# create the directory
os.mkdir(blob_directoryname)

In [11]:
blob_filename = blob_directoryname + blob_hash[2:]
blob_filename

'gitsimplepython/.git/objects/99/72de6ab4626b28c8798abe3624bec5c82ea9c4'

In [12]:
# write the compressed binary content to the file we just created
with open(blob_filename, 'wb') as file:
    file.write(zlib_blob_store)

### What do we have now?

In [13]:
!find gitsimplepython

gitsimplepython
gitsimplepython/.git
gitsimplepython/.git/objects
gitsimplepython/.git/objects/99
gitsimplepython/.git/objects/99/72de6ab4626b28c8798abe3624bec5c82ea9c4


In [14]:
# running `git log` does not yet work though...
!git log

commit 5d6a14b9f552679c839127151ac67e532fa40464
Author: Jes Ford <jesford.data@gmail.com>
Date:   Sun Jun 25 17:03:23 2017 -0600

    Initial commit


### We need a tree

What is a tree?
```
tree <length>\0<mode><name>\0<sha>
```

In [15]:
tree_content = b"100644 README.md\0" + zlib_blob_store
tree_content

b'100644 README.md\x00x\x9cK\xca\xc9OR04d\xc8H\xcd\xc9\xc9W(\xcf/\xcaI\xe1\x02\x00D\x03\x06\x88'

In [16]:
tree_header = "tree {}\0".format(len(tree_content)).encode('utf-8')
tree_header

b'tree 45\x00'

In [17]:
tree_store = tree_header + tree_content
tree_store

b'tree 45\x00100644 README.md\x00x\x9cK\xca\xc9OR04d\xc8H\xcd\xc9\xc9W(\xcf/\xcaI\xe1\x02\x00D\x03\x06\x88'

In [18]:
hash_object = hashlib.sha1(tree_store)
tree_hash = hash_object.hexdigest()
tree_hash

'0f7c91117308c93c365fe11ba0b06e8920790008'

In [19]:
tree_directoryname = "gitsimplepython/.git/objects/{}/".format(tree_hash[0:2])
os.mkdir(tree_directoryname)

In [20]:
tree_filename = tree_directoryname + tree_hash[2:]
tree_filename

'gitsimplepython/.git/objects/0f/7c91117308c93c365fe11ba0b06e8920790008'

In [21]:
zlib_tree_store = zlib.compress(tree_store)
with open(tree_filename, 'wb') as file:
    file.write(zlib_tree_store)

In [22]:
# `git log` still doesn't work... we need a commit
!git log

commit 5d6a14b9f552679c839127151ac67e532fa40464
Author: Jes Ford <jesford.data@gmail.com>
Date:   Sun Jun 25 17:03:23 2017 -0600

    Initial commit


### Create a commit

In [23]:
now = datetime.datetime.now().strftime('%s')  # timestamp in seconds
now

'1498433038'

In [24]:
commit_content = "tree {}\n".format(tree_hash)
commit_content += "author Matt Duff <mattcantstop@github.com> {} +0100\n".format(now)
commit_content += "committer Matt Duff <mattcantstop@github.com> {} +0100\n".format(now)
commit_content += "\nCreate README file\n"

commit_content

'tree 0f7c91117308c93c365fe11ba0b06e8920790008\nauthor Matt Duff <mattcantstop@github.com> 1498433038 +0100\ncommitter Matt Duff <mattcantstop@github.com> 1498433038 +0100\n\nCreate README file\n'

In [25]:
# print formatted nicely
print(commit_content)

tree 0f7c91117308c93c365fe11ba0b06e8920790008
author Matt Duff <mattcantstop@github.com> 1498433038 +0100
committer Matt Duff <mattcantstop@github.com> 1498433038 +0100

Create README file



In [26]:
commit_header = "commit {}\0".format(len(commit_content))
commit_header

'commit 189\x00'

In [27]:
commit_store = commit_header + commit_content
commit_store = commit_store.encode('utf-8')
commit_store

b'commit 189\x00tree 0f7c91117308c93c365fe11ba0b06e8920790008\nauthor Matt Duff <mattcantstop@github.com> 1498433038 +0100\ncommitter Matt Duff <mattcantstop@github.com> 1498433038 +0100\n\nCreate README file\n'

### Create new folder for commit

In [28]:
hash_object = hashlib.sha1(commit_store)
commit_hash = hash_object.hexdigest()
commit_hash

'7df7068289fd437a0a0c2d0dc28293a171f8ef69'

In [29]:
commit_directoryname = "gitsimplepython/.git/objects/{}/".format(commit_hash[0:2])
os.mkdir(commit_directoryname)

In [30]:
commit_filename = commit_directoryname + commit_hash[2:]
commit_filename

'gitsimplepython/.git/objects/7d/f7068289fd437a0a0c2d0dc28293a171f8ef69'

In [31]:
zlib_commit_store = zlib.compress(commit_store)

with open(commit_filename, 'wb') as file:
    file.write(zlib_commit_store)

### What does our directory look like now?

In [32]:
!find gitsimplepython

gitsimplepython
gitsimplepython/.git
gitsimplepython/.git/objects
gitsimplepython/.git/objects/0f
gitsimplepython/.git/objects/0f/7c91117308c93c365fe11ba0b06e8920790008
gitsimplepython/.git/objects/7d
gitsimplepython/.git/objects/7d/f7068289fd437a0a0c2d0dc28293a171f8ef69
gitsimplepython/.git/objects/99
gitsimplepython/.git/objects/99/72de6ab4626b28c8798abe3624bec5c82ea9c4


In [33]:
# still doesn't work...
!git log

commit 5d6a14b9f552679c839127151ac67e532fa40464
Author: Jes Ford <jesford.data@gmail.com>
Date:   Sun Jun 25 17:03:23 2017 -0600

    Initial commit


### We are missing the refs!
Let's add that folder and a master branch reference.

In [34]:
os.mkdir("gitsimplepython/.git/refs/")
os.mkdir("gitsimplepython/.git/refs/heads")

In [35]:
echo_to_file_command = "echo {} > gitsimplepython/.git/refs/heads/master".format(commit_hash)
echo_to_file_command

'echo 7df7068289fd437a0a0c2d0dc28293a171f8ef69 > gitsimplepython/.git/refs/heads/master'

In [36]:
os.system(echo_to_file_command)

0

In [37]:
# check contents of file
!cat gitsimplepython/.git/refs/heads/master

7df7068289fd437a0a0c2d0dc28293a171f8ef69


### Let HEAD know who the current branch is

In [38]:
os.system("echo 'ref: refs/heads/master' > gitsimplepython/.git/HEAD")

0

In [39]:
!find gitsimplepython

gitsimplepython
gitsimplepython/.git
gitsimplepython/.git/HEAD
gitsimplepython/.git/objects
gitsimplepython/.git/objects/0f
gitsimplepython/.git/objects/0f/7c91117308c93c365fe11ba0b06e8920790008
gitsimplepython/.git/objects/7d
gitsimplepython/.git/objects/7d/f7068289fd437a0a0c2d0dc28293a171f8ef69
gitsimplepython/.git/objects/99
gitsimplepython/.git/objects/99/72de6ab4626b28c8798abe3624bec5c82ea9c4
gitsimplepython/.git/refs
gitsimplepython/.git/refs/heads
gitsimplepython/.git/refs/heads/master


In [40]:
!cat gitsimplepython/.git/refs/heads/master

7df7068289fd437a0a0c2d0dc28293a171f8ef69


### Does it work now???

In [41]:
!git log  # oh wait, but this notebook is not in the git directory...

commit 5d6a14b9f552679c839127151ac67e532fa40464
Author: Jes Ford <jesford.data@gmail.com>
Date:   Sun Jun 25 17:03:23 2017 -0600

    Initial commit


In [42]:
!pwd

/Users/jesford/git-implementation-in-python


In [43]:
!cd gitsimplepython/ && git log

commit 7df7068289fd437a0a0c2d0dc28293a171f8ef69
Author: Matt Duff <mattcantstop@github.com>
Date:   Mon Jun 26 00:23:58 2017 +0100

    Create README file


### Hooray!